## Домашнее задание №1

<ol>
    <li>Попробуйте обучить нейронную сеть на Keras с другими параметрами. 
        Опишите в комментарии к уроку - какой результата вы добились от нейросети? Что помогло вам улучшить ее точность?</li>
    <li>Поработайте с документацией Keras.</li>
</ol>

In [22]:
# The full neural network code!
###############################
import numpy as np
import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

# Build the model.
model = Sequential([
  Dense(128, activation='relu', input_shape=(784,)),
  Dense(256, activation='relu'),
  Dense(10, activation='softmax'),
])

# Compile the model.
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=60,
  batch_size=96,
)

# Evaluate the model.
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

# Save the model to disk.
model.save_weights('model.h5')

# Load the model from disk later using:
# model.load_weights('model.h5')

# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]

Epoch 1/60
60000/60000 [==============================] - 8s 141us/step - loss: 0.0572 - accuracy: 0.9802
Epoch 2/60
60000/60000 [==============================] - 6s 105us/step - loss: 0.0271 - accuracy: 0.9909
Epoch 3/60
60000/60000 [==============================] - 7s 122us/step - loss: 0.0204 - accuracy: 0.9931
Epoch 4/60
60000/60000 [==============================] - 7s 122us/step - loss: 0.0169 - accuracy: 0.9943
Epoch 5/60
60000/60000 [==============================] - 7s 124us/step - loss: 0.0141 - accuracy: 0.9952
Epoch 6/60
60000/60000 [==============================] - 7s 113us/step - loss: 0.0125 - accuracy: 0.9957
Epoch 7/60
60000/60000 [==============================] - 7s 114us/step - loss: 0.0114 - accuracy: 0.9960
Epoch 8/60
60000/60000 [==============================] - 7s 112us/step - loss: 0.0093 - accuracy: 0.9968
Epoch 9/60
60000/60000 [==============================] - 7s 112us/step - loss: 0.0091 - accuracy: 0.9967
Epoch 10/60
60000/60000 [=====================

### Выводы

1.

В первую очередь, сделаем самые очевидные изменения в настройках нейронной сети - это изменение количества эпох обучения и размер батчей (epochs=60, batch_size=96). После ряда эксперементов мы получили следующий результат:

Epoch 60/60

60000/60000 [==============================] - 3s 48us/step - loss: 0.0088 - accuracy: 0.9969

10000/10000 [==============================] - 1s 52us/step

[7 2 1 0 4]
[7 2 1 0 4]


Теперь попробуем подобрать loss функцию (при loss=binary_crossentropy) и после ряда экспериментов получили следующий результат:

Epoch 56/60

60000/60000 [==============================] - 5s 80us/step - loss: 0.0012 - accuracy: 0.9996

Epoch 60/60

60000/60000 [==============================] - 5s 80us/step - loss: 0.0021 - accuracy: 0.9993

10000/10000 [==============================] - 1s 67us/step

[7 2 1 0 4]
[7 2 1 0 4]


Увеличение количества нейронов входного и скрытого слоёв (до 128 и 256) позволило резко увеличить accuracy за гораздо меньшее количество эпох, сделать изменение значений loss от эпохи к эпохе более стабильной и получить следующие результаты:


Epoch 60/60

60000/60000 [==============================] - 9s 144us/step - loss: 0.0013 - accuracy: 0.9996

10000/10000 [==============================] - 1s 102us/step

[7 2 1 0 4]
[7 2 1 0 4]


Epoch 200/200

60000/60000 [==============================] - 10s 162us/step - loss: 8.7497e-04 - accuracy: 0.9998

10000/10000 [==============================] - 2s 154us/step

[7 2 1 0 4]
[7 2 1 0 4]

Итак, можно отметить следующее:

   - большое влияние на accuracy и результативность обучения в целом, имеет количество нейронов в слоях (128 и 256);
   - размер батча имеет влияние на скорость обучения 1 эпохи;
   - изменение loss на 'binary_crossentropy' позволило улучшить результат accuracy.



2.

Интересной мне показалась возможность создания своей loss, например:


    def my_loss_fn(y_true, y_pred):
    
        squared_difference = tf.square(y_true - y_pred)
    
        return tf.reduce_mean(squared_difference, axis=-1)  # Note the `axis=-1`

    model.compile(optimizer='adam', loss=my_loss_fn)


И в целом, изучение-просмотр документации по Keras оставило впечатление, что то что мы сейчас делаем cможет и обезьяна, а осмысленное понимание построения графа нейронной сети в лучшем случае придёт не скоро.